In [16]:
import logging
import torch
import io
import re 
from torchtext.utils import download_from_url, extract_archive, unicode_csv_reader
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.vocab import Vocab
from tqdm import tqdm

In [17]:
def _csv_iterator(data_path, ngrams, yield_cls=False):
    male='-1334371409'
    female='1893846424'
    tokenizer = get_tokenizer("basic_english")
    with io.open(data_path, encoding="utf8") as f:
        reader = unicode_csv_reader(f)
        i =-1
        for row in reader:
            segment=None
            gender=None
            i=i+1
            
            if(i > 0) :
                res = None
                try:
                    result = row[6].index(male)  
                    res = re.sub(male, '', row[6])
                    gender=0
                except:
                    res = re.sub(female, '', row[6])
                    gender=1
                    
                row_data = row[1]+ " "+ row[4]+ " "+ row[5]+ " "+ res
                tokens = ' '.join(row_data)
                tokens = tokenizer(tokens)
                if yield_cls:
                    yield gender, ngrams_iterator(tokens, ngrams)
                else:
                    yield ngrams_iterator(tokens, ngrams)

In [23]:
def _create_data_from_iterator(vocab, iterator, include_unk):
    data = []
    labels = []
    with tqdm(unit_scale=0, unit='lines') as t:
        for cls, tokens in iterator:
            #print(cls, tokens)
            if include_unk:
                tokens = torch.tensor([vocab[token] for token in tokens])
            else:
                token_ids = list(filter(lambda x: x is not Vocab.UNK, [vocab[token]
                                        for token in tokens]))
                tokens = torch.tensor(token_ids)
            if len(tokens) == 0:
                logging.info('Row contains no tokens.')
            data.append((cls, tokens))
            labels.append(cls)
            t.update(1)
    return data, set(labels)

In [24]:
class TextClassificationDataset(torch.utils.data.Dataset):
    def __init__(self, vocab, data, labels):
        super(TextClassificationDataset, self).__init__()
        self._data = data
        self._labels = labels
        self._vocab = vocab
    def __getitem__(self, i):
        return self._data[i]

    def __len__(self):
        return len(self._data)

    def __iter__(self):
        for x in self._data:
            yield x

    def get_labels(self):
        return self._labels

    def get_vocab(self):
        return self._vocab

In [56]:
def _setup_datasets( root='/Users/navin/Desktop/test/segment/data/seg/', ngrams=1, vocab=None, include_unk=False):
    #dataset_tar = download_from_url(URLS[dataset_name], root=root)
    #extracted_files = extract_archive(dataset_tar)
    extracted_files =['/Users/navin/Desktop/test/segment/data/seg/train.csv','/Users/navin/Desktop/test/segment/data/seg/test.csv']
    for fname in extracted_files:
        if fname.endswith('train.csv'):
            train_csv_path = fname
            print(train_csv_path)
        if fname.endswith('test.csv'):
            test_csv_path = fname
            print(test_csv_path)

    if vocab is None:
        logging.info('Building Vocab based on {}'.format(train_csv_path))
        vocab = build_vocab_from_iterator(_csv_iterator(train_csv_path, ngrams))
    else:
        if not isinstance(vocab, Vocab):
            raise TypeError("Passed vocabulary is not of type Vocab")
    logging.info('Vocab has {} entries'.format(len(vocab)))
    logging.info('Creating training data')
    train_data, train_labels = _create_data_from_iterator(
        vocab, _csv_iterator(train_csv_path, ngrams, yield_cls=True), include_unk)
    logging.info('Creating testing data')
    test_data, test_labels = _create_data_from_iterator(
        vocab, _csv_iterator(test_csv_path, ngrams, yield_cls=True), include_unk)
    if len(train_labels ^ test_labels) > 0:
        raise ValueError("Training and test labels don't match")
    return (TextClassificationDataset(vocab, train_data, train_labels),
            TextClassificationDataset(vocab, test_data, test_labels))

In [57]:
#train_dataset, test_dataset= TextClassificationDataset()
NGRAMS = 1
train_dataset, test_dataset =_setup_datasets(root='/Users/navin/Desktop/test/segment/data/seg/data/', ngrams=NGRAMS, vocab=None)

1135lines [00:00, 9550.98lines/s]
0lines [00:00, ?lines/s]

/Users/navin/Desktop/test/segment/data/seg/train.csv
/Users/navin/Desktop/test/segment/data/seg/test.csv


1135lines [00:00, 4175.22lines/s]
1000lines [00:00, 3813.49lines/s]


In [67]:
import torch.nn as nn
import torch.nn.functional as F
class TextClassification(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [68]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 1024
NUN_CLASS = len(train_dataset.get_labels())
print("NUN_CLASS",NUN_CLASS,train_dataset.get_labels())
model = TextClassification(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)
BATCH_SIZE = 32


NUN_CLASS 2 {0, 1}


In [69]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [70]:
from torch.utils.data import DataLoader

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        #print(text, offsets, cls)
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [71]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 30
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
sub_train_, sub_valid_ = \
    random_split(train_dataset, [train_len, len(train_dataset) - train_len])

for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

Epoch: 1  | time in 0 minutes, 0 seconds
	Loss: 0.0544(train)	|	Acc: 61.1%(train)
	Loss: 0.0279(valid)	|	Acc: 54.4%(valid)
Epoch: 2  | time in 0 minutes, 0 seconds
	Loss: 0.0260(train)	|	Acc: 57.7%(train)
	Loss: 0.0309(valid)	|	Acc: 45.6%(valid)
Epoch: 3  | time in 0 minutes, 0 seconds
	Loss: 0.0245(train)	|	Acc: 57.2%(train)
	Loss: 0.0188(valid)	|	Acc: 96.5%(valid)
Epoch: 4  | time in 0 minutes, 0 seconds
	Loss: 0.0224(train)	|	Acc: 63.1%(train)
	Loss: 0.0190(valid)	|	Acc: 84.2%(valid)
Epoch: 5  | time in 0 minutes, 0 seconds
	Loss: 0.0208(train)	|	Acc: 61.1%(train)
	Loss: 0.0518(valid)	|	Acc: 45.6%(valid)
Epoch: 6  | time in 0 minutes, 0 seconds
	Loss: 0.0198(train)	|	Acc: 69.7%(train)
	Loss: 0.0295(valid)	|	Acc: 45.6%(valid)
Epoch: 7  | time in 0 minutes, 0 seconds
	Loss: 0.0170(train)	|	Acc: 72.0%(train)
	Loss: 0.0153(valid)	|	Acc: 93.0%(valid)
Epoch: 8  | time in 0 minutes, 0 seconds
	Loss: 0.0157(train)	|	Acc: 73.8%(train)
	Loss: 0.0385(valid)	|	Acc: 54.4%(valid)
Epoch: 9  | time

In [72]:
test_data = DataLoader(test_dataset, batch_size=1, shuffle=True,collate_fn=generate_batch)

In [75]:
vocab

In [101]:
tokenizer = get_tokenizer("basic_english")
y_test=[] 
y_pred =[]
for i, (text, offsets, cls) in enumerate(test_data):
    output = model(text, offsets)
    y_pred.append( output.argmax(1).item())
    y_test.append( cls.data[0].item())
    #print( output.argmax(1).item(),cls.data[0].item())
    

In [104]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
print("Accuracy on Test Data is ", accuracy_score(y_test, y_pred))
confusion_matrix(y_test, y_pred)

Accuracy on Test Data is  0.85


array([[383,  55],
       [ 95, 467]])